## DM lesson4

Dans ce dataset: https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv, chaque ligne correspond à un produit alimentaire mis en vente par un utilisateur.

Objectif: cleaner le dataset.

- On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.
- La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.

Voic une liste d'APIs qui peut vous être utile : https://github.com/public-apis/public-apis (mais vous pouvez en utiliser d'autres si vous le voulez).

In [1]:
import pandas as pd
import requests
import json
import time

In [2]:
products = pd.read_csv('products.csv', sep = ';')

In [3]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
username      200 non-null object
ip_address    200 non-null object
product       200 non-null object
price         200 non-null object
infos         200 non-null object
dtypes: object(5)
memory usage: 7.9+ KB


In [4]:
products.head()

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


## Mettre la colonne *'infos'* sous forme de Boolean en créant une colonne par ingrédient

Commençons par nettoyer la colonne *'infos'* en mettant en minuscule et retirant les signes et mots non pertinents

In [5]:
products['infos'] = products.infos.str.lower()\
                    .str.replace(',','')\
                    .str.replace(':','')\
                    .str.replace('and','')\
                    .str.replace('may','')\
                    .str.replace('contains','')\
                    .str.replace('contain','')\
                    .str.replace('ingredients','')

In [6]:
products.head()

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,peanut fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,mustard fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,sugar egg fish


Utilisation de `get_dummies()` : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html       
Convert categorical variable into dummy/indicator variables

In [7]:
products = products.join(products.infos.str.get_dummies(' '))

In [8]:
products.head()

,username,ip_address,product,price,infos,egg,fish,gluten,milk,mustard,peanut,soja,sugar
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,sugar,0,0,0,0,0,0,0,1
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,peanut fish,0,1,0,0,0,1,0,0
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,mustard fish,0,1,0,0,1,0,0,0
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,gluten,0,0,1,0,0,0,0,0
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,sugar egg fish,1,1,0,0,0,0,0,1


## Créer une colonne de prix unifiés

Pour obtenir la monnaie du pays à partir de l'@ IP, on utilise l'API **'ipapi.co'** qui n'est pas en .json à des fins d'entraînement.

In [14]:
products['price'] = products.price.str.split().str[0]
products['price'] = pd.to_numeric(products.price)

In [15]:
def getCountry(ip):
    
    test=f"https://freegeoip.app/json/{ip}"
    resp = requests.get(test)
    data = resp.json()['country_code']
    
    time.sleep(0.2)
    return data

In [16]:
# Python program to validate an Ip addess . SOURCE= https://www.geeksforgeeks.org/python-program-to-validate-an-ip-address/
  
# re module provides support 
# for regular expressions 
import re 
  
# Make a regular expression 
# for validating an Ip-address 
regex = '''^(25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.( 
            25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)'''
      
# Define a function to validate an Ip addess 
def check(Ip):  
  
    # pass the regular expression 
    # and the string in search() method 
    if(re.search(regex, Ip)):  
        return True

Création d'une colonne ***'valid_ip'*** :

In [18]:
products['valid_ip'] = products['ip_address'].apply(check)

On supprime les lignes ayant *'valid_ip'* à *'NA'*.

In [22]:
products = products.dropna(subset = ['valid_ip'])

On reduit le dataframe pour valider notre solution à petite échelle.

In [22]:
products = products[:50] 

On trouve les pays pour chaque ligne en se basant sur l'IP et on créé un dictionnaire {COUNTRY : local_CURRENCY} :

In [24]:
products['country'] = products['ip_address'].apply(getCountry)

#create a dictionary {COUNTRY : local_CURRENCY}
url_currency= 'http://country.io/currency.json'
res= requests.get(url_currency)
dico_currency = res.json()

On créé une colonne avec la monnaie locale pour chaque ligne en se basant sur le pays :

In [26]:
products['local_currency'] = products['country'].map(dico_currency)

#drop all the rows whose local_currency is NA
products = products.dropna(subset = ['local_currency'])

On créé un dictionnaire pour les taux de change avec comme monnaie de base l'EUR :

In [27]:
url_currency_euro_rate= 'https://api.exchangerate-api.com/v4/latest/EUR'
res2= requests.get(url_currency_euro_rate)
dico_converter = res2.json()['rates']

In [28]:
dico_converter

{'EUR': 1,
 'AED': 4.068761,
 'ARS': 65.90392,
 'AUD': 1.624208,
 'BGN': 1.955803,
 'BRL': 4.654462,
 'BSD': 1.108037,
 'CAD': 1.464947,
 'CHF': 1.096555,
 'CLP': 858.484682,
 'CNY': 7.781921,
 'COP': 3897.318182,
 'CZK': 25.569042,
 'DKK': 7.472131,
 'DOP': 58.606288,
 'EGP': 17.825572,
 'FJD': 2.426174,
 'GBP': 0.855575,
 'GTQ': 8.536539,
 'HKD': 8.670946,
 'HRK': 7.442683,
 'HUF': 335.070554,
 'IDR': 15939.660005,
 'ILS': 3.832073,
 'INR': 79.553202,
 'ISK': 136.241117,
 'JPY': 120.310626,
 'KRW': 1293.95236,
 'KZT': 428.705,
 'MXN': 21.381121,
 'MYR': 4.604291,
 'NOK': 10.097667,
 'NZD': 1.726897,
 'PAB': 1.108037,
 'PEN': 3.738448,
 'PHP': 56.294834,
 'PKR': 171.482,
 'PLN': 4.291514,
 'PYG': 7145.083333,
 'RON': 4.774401,
 'RUB': 70.629441,
 'SAR': 4.155068,
 'SEK': 10.661343,
 'SGD': 1.506962,
 'THB': 33.449047,
 'TRY': 6.331535,
 'TWD': 33.804515,
 'UAH': 26.779808,
 'USD': 1.107716,
 'UYU': 41.702821,
 'VND': 25728.070175,
 'ZAR': 16.349926}

On créé la colonne avec les prix convertis en euros :

In [30]:
products['price_in_euro'] = round(products['price']* products['local_currency'].map(dico_converter),2)
products.head()

,username,ip_address,product,price,infos,egg,fish,gluten,milk,mustard,peanut,soja,sugar,valid_ip,country,currency,local_currency,price_in_euro
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,gluten,0,0,1,0,0,0,0,0,True,US,NaN,USD,387.87
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,sugar egg fish,1,1,0,0,0,0,0,1,True,JP,NaN,JPY,114269.83
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,sugar,0,0,0,0,0,0,0,1,True,MX,NaN,MXN,2385.28
8,epridham8,187.129.113.105,Dried Figs,88.05,sugar milk fish,0,1,0,1,0,0,0,1,True,MX,NaN,MXN,1882.61
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,peanut sugar milk fish,0,1,0,1,0,1,0,1,True,US,NaN,USD,954.02
